#### importing packages

In [19]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

from carriageway import Carriageway
from impact_factor import impact
from load import ll_A, ll_70R, ll_70RT
from reaction import find_bm, find_sf, find_ra, find_rb
# from bridge_specs import box, bearing

In [20]:
span = 50
c_c = 1.5

# live load calculations

#### defining load (vehicles)
load pair contains pair (wheel position, wheel load)

In [21]:
vehicles = [ll_A, ll_70R, ll_70RT]
classA_pair, class70R, class70RT = [list(i.loadpair) for i in vehicles]
loads = [classA_pair, class70R, class70RT]

#### maxBM, maxSF at equal intervals
when each vehicle from the list of vehicles travels along the span, max reactions (BM and SF+/-) at equally divided intervals are calculated and stored in maxBMs, maxSFs_plus, maxSFs_minus.

In [22]:


maxBMs = []
maxSFs_plus = []
maxSFs_minus = []
for i in range(len(loads)):
    maxBM = []
    maxSF_plus = []
    maxSF_minus = []
    for j in range(9):
        at = span / 8 * j
        first_wheel_at = 0
        step = 0.1

        BM = find_bm(span, 0, at)
        SF_plus = find_sf(span, 0, at)
        SF_minus = find_sf(span, 0, at)
        for k in range(int((span + loads[i][-1][0]) / step) + 2):  # '+2' to make sure the load moves all the way to
            # end until it has no effect
            bm = 0
            sf = 0
            for this in loads[i]:
                a, load = this
                pos = -a + first_wheel_at
                bm = bm + find_bm(span, pos, at) * load
                sf = sf + find_sf(span, pos, at) * load
            first_wheel_at += step
            BM = bm if bm > BM else BM
            SF_plus = sf if sf > SF_plus else SF_plus
            SF_minus = sf if sf < SF_minus else SF_minus
        maxBM.append(round(BM, 3))
        maxSF_plus.append(round(SF_plus, 3))
        maxSF_minus.append(round(SF_minus, 3))
    maxBMs.append(maxBM)
    maxSFs_plus.append(maxSF_plus)
    maxSFs_minus.append(maxSF_minus)

#### make a dataframe that contains maxBMs, maxSFs_plus, maxSFs_minus

In [23]:
A = ['MaxBM', 'MaxSF+', 'MaxSF-']
B = ['ClassA', 'Class70RW', 'Class70RT']

# A = ['ClassA', 'Class70RW', 'Class70RT']
# B = ['MaxBM', 'MaxSF+', 'MaxSF-']

iterables = [A, B]
index = pd.MultiIndex.from_product(iterables)

C = []

for i in [maxBMs, maxSFs_plus, maxSFs_minus]:
    C.extend(i)

# for i in range(len(loads)):
#     for j in [maxBMs, maxSFs_plus, maxSFs_minus]:
#         C.append(j[i])

df = pd.DataFrame(C, index=index, columns=[span / 8 * i for i in range(9)])
# print(df.loc[('ClassA', 'MaxSF-')])   ## you can navigate using loc, iloc

new_row = df.loc['MaxSF+'].where(df.loc['MaxSF+'] > abs(df.loc['MaxSF-']), abs(df.loc['MaxSF-']))

new_row.index = pd.MultiIndex.from_product([['MaxSF'], B])
df = pd.concat([df, new_row])

# df.to_excel('outputs/loads.xlsx') # uncomment to save as excel file

*how to read Excel file*

In [24]:
# df = pd.read_excel('outputs/loads.xlsx', index_col=[0, 1])

# ###get index names###
# A = df.index.get_level_values(0).drop_duplicates().to_list()
# B = df.index.get_level_values(1).drop_duplicates().to_list()
# print(A, B)

#### impact factor

In [25]:
IF = [impact(i.name, span) for i in vehicles]
IF

[0.088, 0.088, 0.088]

#### possible combinations for given carriageway width

In [26]:
cwWidth = 6
carriageway = Carriageway(cwWidth)
combinations = carriageway.combinations()
combination_val = [i.get_value() for i in combinations]

#### eccentricity calculation
a combination consists of multiple arrangements gives different eccentricities. maximum eccentricities for each combination calculated.

In [27]:
cw = 6

carriageway = Carriageway(width=cw)
combinations = carriageway.combinations()
list_combinations = []
list_max_e = []

for i in combinations:
    list_combinations.append([i.classA, i.class70Rw, i.class70Rt])
    list_max_e.append(i.max_e())


#### make a dataframe for combinations and their corresponding eccentricities

In [28]:
df1 = pd.DataFrame(list_combinations, columns=['ClassA', 'Class70Rw', 'Class70Rt'],
                   index=[f'comb{i + 1}' for i in range(len(list_combinations))])
df1['MaxEccentricity'] = list_max_e

*export eccentricity as excel file (uncomment)*

In [29]:
# df1.to_excel('outputs/max_e.xlsx')

#### total reactions for given combination
total reactions = sum of (maxreactions at mid span * impact factor * no. of that vehicles)

In [30]:
totalBMs = [(df.loc['MaxBM'][span / 2] * IF).dot(combination_val[i]) for i in range(len(combinations))]
totalSFs = [(df.loc['MaxSF'][span / 2] * IF).dot(combination_val[i]) for i in range(len(combinations))]

In [31]:
# df

In [32]:
df1['TotalBM'] = totalBMs
df1['TotalSF'] = totalSFs

In [33]:
df1

,ClassA,Class70Rw,Class70Rt,MaxEccentricity,TotalBM,TotalSF
comb1,2,0,0,-3.739699e-16,972.6024,30.828512
comb2,0,1,0,-4.050000e-01,957.2112,34.982112
comb3,0,0,1,-3.500000e-01,734.1004,27.947920


# loads on peir

In [34]:
dfSF = pd.DataFrame()
dfBM = pd.DataFrame()
for veh in vehicles:
    dummy_load = list(veh.loadpair)
    no = 2
    nose_dist = 20
    if veh.name == 'Class 70RT':
        nose_dist = 90
        no = 1
    elif veh.name == 'Class 70RW':
        nose_dist = 31.52

    final_load = list(dummy_load)
    for i in range(no - 1):
        delta = dummy_load[-1][0] + nose_dist
        final_load.extend([(i[0] + delta, i[1]) for i in dummy_load])

    max_sum_at = 0
    sumlr = 0
    difflr = 0
    nleft = 0
    nright = 0

    head_at = 0
    step = 0.01
    for i in range(int((span * 2 + final_load[-1][0]) / step)):
        left = 0
        right = 0
        for j in final_load:
            rb = find_rb(span, head_at - j[0], c_c / 2, c_c / 2) * j[1]
            ra = find_ra(span, head_at - j[0] - (span + c_c), c_c / 2, c_c / 2) * j[1]
            temp_load = 0
            if head_at - j[0] == span + c_c / 2:
                #                 print('hjgjh')
                temp_load = (j[1]) / 2
                ra = find_ra(span, head_at - j[0] - (span + c_c), c_c / 2, c_c / 2) * temp_load
                rb = find_rb(span, head_at - j[0], c_c / 2, c_c / 2) * temp_load
            left += rb
            right += ra

        if left + right > sumlr:
            sumlr = left + right
            dfSF.loc[f'{veh.name}', 'RL'], dfSF.loc[f'{veh.name}', 'RR'], dfSF.loc[
                f'{veh.name}', 'at'], dfSF.loc[f'{veh.name}', 'sum']= left, right, head_at, left + right

        if abs(left - right) > abs(difflr):
            difflr = left - right
            dfBM.loc[f'{veh.name}', 'RL'], dfBM.loc[f'{veh.name}', 'RR'], dfBM.loc[
                f'{veh.name}', 'at'], dfBM.loc[f'{veh.name}', 'sum'] = left, right, head_at, left + right
        head_at = round(head_at + step, 5)

In [35]:
total_frame = pd.concat([dfSF, dfBM], keys=['MaxSF', 'MaxBM'])
total_frame.to_excel('outputs/Live_Loads_Seismic.xlsx')
total_frame

RL        RR      at        sum
MaxSF Class A     240.2016  454.5144   69.57   694.7160
      Class 70RW  219.8760  911.7240   64.19  1131.6000
      Class 70RT  344.8160  349.3660   52.71   694.1820
MaxBM Class A     508.7832    0.0000   50.74   508.7832
      Class 70RW    0.0000  952.1040  109.08   952.1040
      Class 70RT  678.3700    0.0000   50.74   678.3700